<a href="https://colab.research.google.com/github/aruaru0/llm-hands-on/blob/main/llm_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colabの使い方

1. [https://colab.google/](https://colab.google/)にアクセス→"New Notebook"または「新規作成」を選択
2. Googleアカウントでのログインを求められた場合はログインする
3. コードブロック（セル）にPythonコードを書いてShift-Enterで実行

## 練習（１）
とりあえず、"hello"と表示



In [ ]:
print("hello")

hello


# LLMハンズオン用コード

## 実行前の注意点
**実行前に必ず「ランタイム」→「ランタイムのタイプを変更」で<font color=red>"T4 GPU"</font>を選択しておくこと！**

## 便利な設定
「ツール」→「設定」→「AIアシスタント」→「生成AIによるコード補完を表示」

<font color="red">**「生成AI機能の使用に同意している」のチェックも必要**</font>


# 1. 前準備 ==============

## ⭐️⭐️　今回は、デコーダーモデルを使って演習を行います

## ①必要なパッケージをインポートする
以下の２つのパッケージから、必要となる機能をインポートします
* transformerからAutoModelForCasualLMとAutoTokenizer
* torchとtorch.nn.functional

`as F`は、**Fという名前**で読み込むことを指定する記述方法です

<font color="blue">下記のコードを入力したら、コードセル左の実行ボタンをクリック、またはshift+enterでコードを実行してください。</font>

In [ ]:
# パッケージのインポート
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import torch.nn.functional as F

## ②モデルを読み込む

モデルは、トークン化するための**AutoTokenizer**と、デコーダー本体の**AutoModelForCausalLM**に分かれています。今回は、モデルは楽天のRakutenAI-2.0-miniを利用します。  

* tokenizerとmodelは一致していないとうまく動きません（<font color="red">トークナイザーで出力するトークンIDとモデルのトークンIDが一致している必要があります</font>）
* llamaなどのモデルはHuggingface.ioのサイトにログインし認証する必要があるので注意

**学習済みモデル**を使いたいので`from_pretrained`（訓練済みの指示）をつけています。

また、モデルはdeviceをautoにしています（GPUがあれば自動的にGPUに転送されます）

In [ ]:
model_path = "Rakuten/RakutenAI-2.0-mini"
# model_path = "sbintuitions/sarashina2.2-0.5b"

# トークナイザー（トークン化するためのライブラリ）
tokenizer = AutoTokenizer.from_pretrained(model_path)
# デコーダーモデルを読み込む
model = AutoModelForCausalLM.from_pretrained(model_path, torch_dtype="auto", device_map="auto")
# 評価モードへ
model.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/755k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/712 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.07G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(48000, 2048)
    (layers): ModuleList(
      (0-21): 22 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): MistralRMSNorm((2048,), eps=1e-05)
  

# 2. 文字列とトークンIDの相互変換 ==============

ここでは、文字列がどのような「トークンIDになるのか？」を確認してみます

このために、文字列→トークンIDと、変換されたトークンIDから文字列への変換を行ってみます

## 1. 文字列からトークンID

* 先ほどロードしたtokenizerに文字列を入力するとトークンIDに変換されます

## 2. トークンIDから単語

* tokenizerには、トークンIDを文字列に直す関数も用意されています`convert_ids_to_tokens()`

※ トークンには特殊なものがあります。例えば`<s>`は文章の開始を表すトークンです（変換時に自動的に付加されます）


## ③文字列→トークンID

In [ ]:
text = "this is a pen."
input = tokenizer(text)
print(input)

{'input_ids': [1, 894, 349, 264, 4969, 28723], 'attention_mask': [1, 1, 1, 1, 1, 1]}


## ④トークンID→単語

In [ ]:
# tokenをデコードして文字列に直す
tokenizer.convert_ids_to_tokens(input['input_ids'])

['<s>', 'this', '▁is', '▁a', '▁pen', '.']

# 3. embedding（トークンのベクトル表現）を確認する  ==============

ここでは、トークンIDがベクトル表現になるのを確認してみます

ベクトル表現を取り出すには、modelのembedding（埋め込み層）の出力を取り出す必要があります。

modelから埋め込み層だけ取り出す関数が`get_input_embeddings()`として用意されていますのでこれを使います。

1. tokenizerで文字列をトークンID（'input_ids'）に変換する
2. 埋め込み層だけ取り出す
3. トークンID（'input_ids'）を埋め込み層に入力するとベクトル表現が出力される
4. 受け取った内容を表示する（大きいので、**ベクトルの形状と先頭の一部**を確認する）

### ポイント
* 出力は（1, トークン数, ベクトルサイズ）の形式で格納（※先頭の１はバッチサイズ）
* 今回のモデルの<font color="red">**埋め込みベクトルの次元数は？**</font>

## ⑤ベクトル表現を取り出す

In [ ]:
# 文字列を入力しトークンIDに変換
text = "Hello, how are you?"
inputs = tokenizer(text, return_tensors="pt") #ptはPytorch Tensor型

# 埋め込み層だけ抜き出す
embedding_layer = model.get_input_embeddings()

# 埋め込み層にトークンIDを入力して、埋め込みベクトル（embedding_output）に変換
input_ids = inputs["input_ids"].to(model.device)  # GPU/CPUのデバイスに合わせる
embedding_output = embedding_layer(input_ids)

# 埋め込みベクトルの形状を表示
print("入力トークンID ", input_ids)
print("入力トークン   ", tokenizer.convert_ids_to_tokens(input_ids[0]))
print("ベクトルの形状 ", embedding_output.shape)     # ベクトルの形状を確認(1, sequence_length, hidden_size)
print("ベクトルの中身 ", embedding_output[0,1,:100]) # helloのベクトルの先頭100個を表示

入力トークンID  tensor([[    1, 16230, 28725,   910,   460,   368, 28804]], device='cuda:0')
入力トークン    ['<s>', 'Hello', ',', '▁how', '▁are', '▁you', '?']
ベクトルの形状  torch.Size([1, 7, 2048])
ベクトルの中身  tensor([-0.0175, -0.0247, -0.0112, -0.0225, -0.0063, -0.0019, -0.0089, -0.0017,
        -0.0271,  0.0178, -0.0004,  0.0098, -0.0198, -0.0047,  0.0069,  0.0228,
         0.0474, -0.0177, -0.0082, -0.0179, -0.0045, -0.0003, -0.0115, -0.0023,
         0.0075,  0.0063,  0.0190, -0.0071,  0.0023,  0.0332,  0.0117, -0.0033,
         0.0123,  0.0074, -0.0060,  0.0007,  0.0132, -0.0110,  0.0082, -0.0201,
         0.0530,  0.0254,  0.0164,  0.0020,  0.0284, -0.0101,  0.0344,  0.0080,
        -0.0292, -0.0072, -0.0197, -0.0047,  0.0101,  0.0020, -0.0200, -0.0042,
         0.0132, -0.0080,  0.0266,  0.0199, -0.0330,  0.0023, -0.0122,  0.0342,
         0.0107,  0.0138,  0.0072,  0.0113, -0.0060,  0.0048,  0.0071, -0.0081,
         0.0013, -0.0148, -0.0070,  0.0449,  0.0236,  0.0242,  0.0068,  0.0332,
        -

# 3. デコーダーを動かしてみる ==============
デコーダーモデルを実際に動かしてみます

* デコーダの生の出力（<font color="red">確率分布</font>）を確認してみます
* 入力した文字列の次の文字が予測できていることを確認します
  - ここでは、予測結果の上位５つを表示してみます。

## ポイント
1. モデルをGPUに配置しているので、変換したトークンIDもGPUに移動させる必要があります（`to(model.device)`）
1. 予測スコアはモデルの出力の`logits`に格納されています
1. 最後のトークンの予測スコアだけ取り出すので末尾を取り出します（pythonでは<font color="red">-1</font>で後ろから参照できます）
1. `softmax(..., dim=-1)`で行ごとに正規化が行われます（dim=-1で行単位を指定）
1. 出力の形状から、<font color="red">**入力可能なトークンIDの種類**</font>が分かります。確認してみてください
1. `decode`を使うことで、トークンIDを文字に戻しています

## ⑥次の文字を予測してみる

In [ ]:
# 入力文
input_text = "日本で一番高い山は"

# トークン化
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

# モデルでの予測
with torch.no_grad():
    # モデルを使用して出力を得る
    outputs = model(input_ids)
    # モデルの出力（生のスコア）を取得
    logits = outputs.logits
    # 出力の末尾が最後のトークンに対応するスコアなので、それだけ抜き出す
    # (1, sequence_length, 重み)
    last_token_logits = logits[:, -1, :]
    # softmaxを適用して確率分布を得る
    softmax_probs = F.softmax(last_token_logits, dim=-1)

print(f"入力テキスト: {input_text} : {input_ids}")
print(f"出力の形状: {logits.shape}")

# 上位k個を取得
k = 5
top_k_values, top_k_indices = torch.topk(softmax_probs, k=5, dim=-1)

# 結果の表示
for prob, predicted_token_id in zip(top_k_values[0], top_k_indices[0]):
  predicted_token = tokenizer.decode(predicted_token_id, skip_special_tokens=True)
  print("-"*40)
  print(f"softmax確率: {prob*100:.2f}%")
  print(f"  予測されたトークンID: {predicted_token_id.item()}")
  print(f"  予測されたトークン: {predicted_token}")

入力テキスト: 日本で一番高い山は : tensor([[    1, 35933, 40134, 33371, 45042]], device='cuda:0')
出力の形状: torch.Size([1, 5, 48000])
----------------------------------------
softmax確率: 45.25%
  予測されたトークンID: 36831
  予測されたトークン: 富士
----------------------------------------
softmax確率: 10.75%
  予測されたトークンID: 29771
  予測されたトークン: ？
----------------------------------------
softmax確率: 6.12%
  予測されたトークンID: 29041
  予測されたトークン: 、
----------------------------------------
softmax確率: 2.27%
  予測されたトークンID: 28705
  予測されたトークン:  
----------------------------------------
softmax確率: 1.99%
  予測されたトークンID: 38851
  予測されたトークン: どこ


## ⑦デコーダーを連続で動かしてみる

予測した文字列を追加しながら、連続で動かして文章が生成されていくのを確認してみます。
* 先ほどのプログラムをループに変えます
* `argmax`で一番スコアの高いものを選択するように変更します

In [ ]:
# 入力文
input_text = "昨日は雨でした。傘を"

# 10回ループする
for _ in range(10):
  # モデルでの予測
  with torch.no_grad():
      # トークン化
      input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)

      # モデルを使用して出力を得る
      outputs = model(input_ids)

      # ログits（モデルの出力生のスコア）を取得
      logits = outputs.logits

      # 最後のトークンに対応するスコアを取り出す
      last_token_logits = logits[:, -1, :].cpu()

      # softmaxを適用して確率分布を得る
      softmax_probs = F.softmax(last_token_logits, dim=-1)

      # 予測されたトークンIDを取得
      predicted_token_id = torch.argmax(softmax_probs, dim=-1)

      predicted_token = tokenizer.decode(predicted_token_id, skip_special_tokens=True)
      print(predicted_token_id, tokenizer.eos_token_id)

      # 末尾に予測を足してループ
      input_text += predicted_token
      print(input_text)

# 4. Generateを使う例 ==============

ここまで自分で１文字つづ予測し追加していましたが、これを自動で行う`generate()`という関数があるのでこれを使ってみます。

`generate`の引数
* `max_new_tokens`　最大のトークン数
* `do_sample`　サンプル戦略を有効にする（trueの場合モデルは確率分布に基づいてランダムにトークンを選択）
* `temperature`　どの程度の確率で選ぶか（小さくするほど確率の高いものを選択するようになる）
* `top_p`　0.9の場合、最も確率が高い90%のトークンの中から次を選択する
* `repetition_penalty`　同じトークンを繰り返し生成することを抑制するためのペナルティ値（大きくするほど繰り返しが抑制される）

## ⑧デコーダの一般的な使い方

In [ ]:
inputs = tokenizer("この物語は、", return_tensors="pt").to(model.device)
with torch.no_grad():
    tokens = model.generate(
        **inputs,
        max_new_tokens=32,
        do_sample=True, # サンプリング戦略を有効にする（tempertureなどが有効になる）
        temperature=0.7,
        top_p=0.9,
        repetition_penalty=1.05,
        pad_token_id=tokenizer.pad_token_id,
    )

output = tokenizer.decode(tokens[0], skip_special_tokens=True)
print(output)

この物語は、主人公である主人公が、自分を見失っている時に出会った人物から、自分の本当の姿に気づくことから始まる。
主人公は、16歳にして、母親を亡くし、
